In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt

riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(False)

In [3]:
client = MongoClient()
loladb = client.loladb73
summoners_collection = loladb.summoners
matches_collection = loladb.matches
print summoners_collection.count()
print matches_collection.count()

2829
30993


In [4]:
champions = riotapi.get_champions()
dict_id2idx = dict()
dict_idx2id = dict()
dict_id2key = dict()
for a_idx, a_champion in enumerate(champions):
    champ_id = a_champion.id
    champ_key = a_champion.key
    dict_id2idx[champ_id] = a_idx
    dict_idx2id[a_idx] = champ_id
    dict_id2key[champ_id] = champ_key
    
#print dict_id2key[dict_idx2id[132]]
#print dict_id2key

def GetIdxFromId(a_id):
    return dict_id2idx[a_id]

def GetIdFromIdx(a_idx):
    return dict_idx2id[a_idx]

def GetNameFromId(a_id):
    return dict_id2key[a_id]

In [5]:
def AddToDict(a_dict, a_key, a_value):
    if a_key in a_dict.keys():
        a_dict[a_key] += a_value
    else:
        a_dict[a_key] = a_value

In [22]:
#统计最常出现的英雄二人组合
#从单个英雄开始，再到双英雄组合，三英雄组合的尝试
champion_single_win = {}
champion_single_num = {}
champion_double_win = {}
champion_double_num = {}
champion_against_win = {}
champion_against_num = {}
#统计三英雄组合需要基于双英雄组合的结果，仅记录包含100场以上双英雄的组合
champion_triple_win = {}
champion_triple_num = {}

def AddAgainstChamp(a_champions, b_champions, isWinning):
    for a_champ in a_champions:
        for b_champ in b_champions:
            switched = False
            double_key = a_champ + '_' + b_champ
            if a_champ > b_champ:
                switched = True
                double_key = b_champ + '_' + a_champ
            AddToDict(champion_against_num, double_key, 1)
            if isWinning != switched:
                AddToDict(champion_against_win, double_key, 1)

def AddDoubleChamp(team_champions, isWinning):
    for a_champ in team_champions:
        for another_champ in team_champions:
            if a_champ != another_champ:
                double_key = a_champ + '_' + another_champ
                if a_champ > another_champ:
                    double_key = another_champ + '_' + a_champ

                AddToDict(champion_double_num, double_key, 1)
                if isWinning:
                    AddToDict(champion_double_win, double_key, 1)
                
                    
for a_idx, a_match in enumerate(matches_collection.find()):
    #check the champs appear in this game
    if a_match['matchDuration'] <= 1200:
        continue
        
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    if winning_side == -1:
        continue
        
    participants = a_match['participants']
    blue_champions = []
    red_champions = []
    for a_participant in participants:
        champion_name = GetNameFromId(a_participant['championId'])
        champion_index = a_participant['participantId']
        champion_side = 0
        if champion_index > 5:
            champion_side = 1
            red_champions.append(champion_name)
        else:
            blue_champions.append(champion_name)
        AddToDict(champion_single_num, champion_name, 1)
        if winning_side == champion_side:
            AddToDict(champion_single_win, champion_name, 1)
            
    AddDoubleChamp(blue_champions, winning_side == 0)
    AddDoubleChamp(red_champions, winning_side == 1)
    AddAgainstChamp(blue_champions, red_champions, winning_side == 0)
    AddAgainstChamp(red_champions, blue_champions, winning_side == 1)
    
    if a_idx % 1000 == 0:
        print a_idx

0
1000
3000
4000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
28000
29000
30000


In [12]:
#仅保留超过一定数目的双人组
DOUBLE_LIMIT = 100
pair_to_delete = []
keep_double_num = champion_double_num.copy()
for a_pair in keep_double_num:
    if keep_double_num[a_pair] < DOUBLE_LIMIT:
        pair_to_delete.append(a_pair)
        
for a_pair in pair_to_delete:
    del keep_double_num[a_pair]
print len(keep_double_num)

2373


In [15]:
#统计三人组                    
def AddTripleChamp(team_champions, isWinning):
    for a_champ in team_champions:
        for another_champ in team_champions:
            if a_champ != another_champ:
                double_key = a_champ + '_' + another_champ
                if a_champ > another_champ:
                    double_key = another_champ + '_' + a_champ
                    
                if double_key in keep_double_num.keys():
                    for third_champ in team_champions:
                        if third_champ == a_champ or third_champ == another_champ:
                            continue
                        key_one = double_key.split('_')[0]
                        key_two = double_key.split('_')[1]
                        triple_key = ''
                        if third_champ < key_one:
                            triple_key = third_champ + '_' + key_one + '_' + key_two
                        elif third_champ < key_two:
                            triple_key = key_one + '_' + third_champ + '_' + key_two
                        else:
                            triple_key = key_one + '_' + key_two + '_' + third_champ 
                        AddToDict(champion_triple_num, triple_key, 1)
                        if isWinning:
                            AddToDict(champion_triple_win, triple_key, 1)

for a_idx, a_match in enumerate(matches_collection.find()):
    #check the champs appear in this game
    if a_match['matchDuration'] <= 1200:
        continue
        
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    if winning_side == -1:
        continue
        
    participants = a_match['participants']
    blue_champions = []
    red_champions = []
    for a_participant in participants:
        champion_name = GetNameFromId(a_participant['championId'])
        champion_index = a_participant['participantId']
        champion_side = 0
        if champion_index > 5:
            champion_side = 1
            red_champions.append(champion_name)
        else:
            blue_champions.append(champion_name)
            
    AddTripleChamp(blue_champions, winning_side == 0)
    AddTripleChamp(red_champions, winning_side == 1)
    
    if a_idx % 200 == 0:
        print a_idx

0
200
400
600
800
1000
1200
1400
1600
1800
2200
2400
2600
2800
3000
3400
3800
4000
4200
4400
4600
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24800
25000
25200
25400
25800
26000
26200
26400
26600
26800
27200
27400
27600
28000
28200
28400
28600
28800
29000
29200
29400
29600
29800
30000
30200
30400
30600
30800


In [16]:
champion_triple_winrate = {}
for triple_idx, a_triple in enumerate(champion_triple_num):
    a_win = 0
    if a_triple in champion_triple_win.keys():
        a_win = champion_triple_win[a_triple]
    a_num = champion_triple_num[a_triple]
    a_winrate = a_win/float(a_num)
    champion_triple_winrate[a_triple] = a_winrate
    if triple_idx % 500 == 0:
        print triple_idx

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
8500

In [6]:
champion_single_winrate = {}
for each_champion in champion_single_num:
    a_win = champion_single_win[each_champion]
    a_num = champion_single_num[each_champion]
    a_winrate = a_win / float(a_num)
    champion_single_winrate[each_champion] = a_winrate

In [23]:
champion_against_winrate = {}
for pair_idx, a_pair in enumerate(champion_against_num):
    a_win = 0
    if a_pair in champion_against_win.keys():
        a_win = champion_against_win[a_pair]
    a_num = champion_against_num[a_pair]
    a_winrate = a_win / float(a_num)
    champion_against_winrate[a_pair] = a_winrate
    if pair_idx % 500 == 0:
        print pair_idx

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500


In [38]:
champion_double_winrate = {}
for pair_idx, a_pair in enumerate(champion_double_num):
    a_win = 0
    if a_pair in champion_double_win.keys():
        a_win = champion_double_win[a_pair]
    a_num = champion_double_num[a_pair]
    a_winrate = a_win / float(a_num)
    champion_double_winrate[a_pair] = a_winrate
    if pair_idx % 500 == 0:
        print pair_idx

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000


In [18]:
import operator
sorted_winrate = sorted(champion_single_winrate.items(), key=operator.itemgetter(1))

NameError: name 'champion_single_winrate' is not defined

In [39]:
sorted_double_winrate = sorted(champion_double_winrate.items(), key=operator.itemgetter(1))

In [24]:
sorted_against_winrate = sorted(champion_against_winrate.items(), key=operator.itemgetter(1))

In [19]:
import operator
sorted_triple_winrate = sorted(champion_triple_winrate.items(), key=operator.itemgetter(1))

In [12]:
for each_champion in sorted_winrate:
    print (each_champion, champion_single_winrate[each_champion[0]], champion_single_num[each_champion[0]])

 ((u'Udyr', 0.4), 0.4, 20)
((u'Chogath', 0.4074074074074074), 0.4074074074074074, 108)
((u'Sejuani', 0.4175824175824176), 0.4175824175824176, 91)
((u'Viktor', 0.4268447837150127), 0.4268447837150127, 1572)
((u'Amumu', 0.4304932735426009), 0.4304932735426009, 223)
((u'Diana', 0.4312267657992565), 0.4312267657992565, 269)
((u'Garen', 0.431924882629108), 0.431924882629108, 213)
((u'Tristana', 0.43646408839779005), 0.43646408839779005, 362)
((u'Nasus', 0.44155844155844154), 0.44155844155844154, 308)
((u'Kindred', 0.4426229508196721), 0.4426229508196721, 183)
((u'DrMundo', 0.4427083333333333), 0.4427083333333333, 192)
((u'Veigar', 0.44471744471744473), 0.44471744471744473, 407)
((u'Azir', 0.4454828660436137), 0.4454828660436137, 321)
((u'Galio', 0.45), 0.45, 80)
((u'MasterYi', 0.4505672609400324), 0.4505672609400324, 617)
((u'Lissandra', 0.45306859205776173), 0.45306859205776173, 554)
((u'Lux', 0.4557291666666667), 0.4557291666666667, 384)
((u'Morgana', 0.4580098800282287), 0.45800988002822

In [40]:
import csv
with open("double_winrate.csv", 'wb') as csvfile:
    csvfile.write(u'\ufeff'.encode('utf8'))
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(["Name1", "Name2", "Win Rate", "Number"])

    for each_champion in sorted_double_winrate:
        csvwriter.writerow([each_champion[0].split('_')[0], each_champion[0].split('_')[1], champion_double_winrate[each_champion[0]], champion_double_num[each_champion[0]]])

In [27]:
with open("against_winrate.csv", 'wb') as csvfile:
    csvfile.write(u'\ufeff'.encode('utf8'))
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(["Name1", "Name2", "Win Rate", "Number"])

    for each_champion in sorted_against_winrate:
        name1 = each_champion[0].split('_')[0]
        name2 = each_champion[0].split('_')[1]
        each_num = champion_against_num[each_champion[0]]
        if each_num < 100:
            continue
        csvwriter.writerow([name1, name2, champion_against_winrate[each_champion[0]], each_num])

In [35]:
 for each_champion in sorted_double_winrate:
        print each_champion[0]
        break

Akali_Annie


In [28]:
for a_win in champion_double_num:
    print a_win
    break

Akali_Annie


In [8]:
print champion_double_num['Akali_Annie']

34


In [21]:
import csv
with open("triple_winrate.csv", 'wb') as csvfile:
    csvfile.write(u'\ufeff'.encode('utf8'))
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(["Name1", "Name2", "Name3", "Win Rate", "Number"])
    for each_triple in sorted_triple_winrate:
        each_winrate = champion_triple_winrate[each_triple[0]]
        each_num = champion_triple_num[each_triple[0]]
        names = each_triple[0].split('_')
        if each_winrate > 0 and each_num > 100:
            csvwriter.writerow([names[0], names[1], names[2], each_winrate, each_num])